In [30]:
# question 1: # Inductive reasoning is a type of logical thinking that involves making generalizations based on specific observations or cases. 
# However , deductive reasoning is making specific conclusions based off general principles.
# An example of inductive reasoning could be assuming that its going to snow during the winter because it has every previous year.
# An example of dedcutive reasoning could be assuming rich people have large houses.

In [31]:
# question 2
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from typing import List
from typing import Tuple
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [32]:
heart_disease_df = pd.read_csv('heartdisease.csv')
heart_disease_df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb3 in position 11: invalid start byte

In [ ]:
#question 2: preprocess your dataset 
def preprocess_data(df: pd.DataFrame, column_names: List[str]) -> pd.DataFrame:

    df.columns = column_names

    df['ca'] = pd.to_numeric(df['ca'], errors='coerce')
    df['thal'] = pd.to_numeric(df['thal'], errors='coerce')

    df['ca'] = df['ca'].fillna(df['ca'].median())
    df['thal'] = df['thal'].fillna(df['thal'].median())

    df['num'] = np.where(df['num'] >= 2, 1, df['num'])

    return df
df = preprocess_data(df, column_names)
df.head()

In [ ]:
# question 3: creating a decition tree model 
#Class weight-balanced gives more weight to the minority class, helping the model deal with class imbalances and generally 
#increasing recall for the minority class, which in this case is 1. Keeping max_features at 10 decreases the overfitting because 
#only a subset of features are considered for each split. Splitter='best' enables the model at each node to select the best split, 
#hence being more accurate on the data. Limiting max_depth to 7 can prevent the tree from being too deep; it also prevents overly specific
#splits, which improves generalization and, by extension, increases recall because a class is less likely to be missed. 
#Finally, reproducibility is guaranteed with random_state=42: the results are reproducible and more interpretable. 
#These changes altogether enhance the capability of the model to recognize positive examples in all
def train_and_evaluate_model(df: pd.DataFrame) -> Tuple[str, DecisionTreeClassifier]:
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = DecisionTreeClassifier(
        class_weight='balanced',
        max_features=10, 
        splitter='best',  
        max_depth=7, 
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred)
    return report, model
report, model = train_and_evaluate_model(df)
print(report)

In [ ]:
# question 4: creating a random forest model 

def train_and_evaluate_rf_model(df: pd.DataFrame) -> Tuple[float, RandomForestClassifier]:
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    rf = RandomForestClassifier(
        n_estimators=300,         
        max_depth=12,            
        min_samples_split=3,      
        min_samples_leaf=1,       
        class_weight='balanced', 
        random_state=42
    )
    rf.fit(X_train, y_train)
    score = rf.score(X_test, y_test)
    ConfusionMatrixDisplay.from_estimator(rf, X_test, y_test)
    return score, rf
score, rf = train_and_evaluate_rf_model(df)

In [ ]:
# question 5 I used two of the same parameters as previous models, but I also added gamma=1 that controls just how much the model is 
#"allowed" to split the trees. It acts like a penalty for doing splits. In other words, when the split results in a small improvement 
#in model accuracy, the value of gamma makes it harder for the tree to split. That is, here gamma =1 signifies that the model will make 
#a split only when the split improves the prediction accuracy at least by 1 unit.

def train_and_evaluate_xgb_model(df: pd.DataFrame) -> Tuple[XGBClassifier, pd.Series]:
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    xgb = XGBClassifier(
        n_estimators=300,          
        max_depth=10,
        gamma=1
    )
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    ConfusionMatrixDisplay.from_estimator(xgb, X_test, y_test)
    return xgb, y_pred
xgb, y_pred = train_and_evaluate_xgb_model(df)